In [7]:
from os import listdir
from os.path import isfile, join
from PIL import Image

import random
import math
import time
import numpy as np
import cv2
import os
import shutil

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import torch.nn as nn
import torch.nn.functional as F

In [8]:
widthSteering = heightSteering = 140

widthRoad = 600
heightRoad = 200

modelsPath = 'Models'
modelName = 'bestModel_dayOnly_MSEof6.4e-0.5.torch'
pathToModel = f'{modelsPath}/{modelName}'

framesPath = 'Frames'
labeledFrames = 'LabeledFrames'
wheelFrames = 'WheelFrames'

#road crop area
x1R = 400
x2R = 600
y1R = 630
y2R = 1280

#road image size
widthRoad  = y2R - y1R
heightRoad = x2R - x1R

#wheel crop area
x1W = 782
x2W = 962
y1W = 903
y2W = 1043

#wheel image size
widthWheel  = y2W - y1W
heightWheel = x2W-x1W


In [9]:
exist = os.path.exists(modelsPath)
if exist:
    print('Models Path Directoty Exists')
else:
    print('Directory Does not Exists')

exist = os.path.exists(framesPath)
if exist:
    print("Frames Directoty Exists")
else:
    print("Frames Directory Does Not Exists")


exist = os.path.exists(labeledFrames)
if not exist:
    os.makedirs(labeledFrames)
    print("Labled Frames Directoty Created")
else:
    print('Labled Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    #shutil.rmtree(labeledFrames)
    #os.makedirs(labeledFrames)

exist = os.path.exists(wheelFrames)
if not exist:
    os.makedirs(wheelFrames)
    print("Wheel Frames Directoty Created")
else:
    print('Wheel Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    #shutil.rmtree(labeledFrames)
    #os.makedirs(labeledFrames)

Models Path Directoty Exists
Frames Directoty Exists
Labled Frames Directory Exists
Wheel Frames Directory Exists


In [10]:
startingFrames = [f for f in listdir(framesPath) if isfile(join(framesPath, f))]

In [11]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(512, 128)
        
        self.branch_a1 = nn.Linear(128, 32)
        self.branch_a2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc(x))

        a = F.leaky_relu(self.branch_a1(x))
        
        out1 = self.branch_a2(a)
        
        return out1

In [12]:
#load net
#To Load Pretrained Weights:   weights='ResNet18_Weights.DEFAULT'
resnet18 = torchvision.models.resnet18()
resnet18.fc = nn.Identity()
net_add=net()
model = nn.Sequential(resnet18, net_add)

# Set device GPU or CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

# load model to GPU
model = model.to(device)

model.load_state_dict(torch.load(pathToModel))

<All keys matched successfully>

In [17]:
def run():
    i = 0
    for file in startingFrames:
        i=i+1
        if i%100 == 0:
            print(f'Frame No: {i}')
        #dont want the '.jpg' element of the file name in the file name
        noExtension = file[:-4]

        filePath = f'{framesPath}/{file}'

        #load image
        wholeFrame = cv2.imread(filePath)

        #crop to steering wheel logo and transform image to black and white but in RGB color space
        crop = wholeFrame[x1W:x2W, y1W:y2W]

        grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)
        backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)

        

        transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((heightWheel,widthWheel)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
        wheel=transformImg(np.array(backtorgb))

        wheel = wheel.to(device).unsqueeze(0)

        with torch.no_grad():
            prediction = model(wheel)  # Run net

        detectedAngle = prediction.data.cpu().numpy()[0][0]

        realAngle = (180*detectedAngle)-90

        cv2.imwrite(f'{wheelFrames}/wheelCrop_angleNorm_{detectedAngle}_angleReal_{realAngle}.jpg',backtorgb)

        crop = wholeFrame[x1R:x2R, y1R:y2R]

        cv2.imwrite(f'{labeledFrames}/{detectedAngle}_degrees_roadcrop_{noExtension}.jpg',crop)



In [18]:
run()

Frame No: 100
Frame No: 200
Frame No: 300
Frame No: 400
Frame No: 500
Frame No: 600
Frame No: 700
Frame No: 800
Frame No: 900
Frame No: 1000
Frame No: 1100
Frame No: 1200
Frame No: 1300
Frame No: 1400
Frame No: 1500
Frame No: 1600
Frame No: 1700
Frame No: 1800
Frame No: 1900
Frame No: 2000
Frame No: 2100
Frame No: 2200
Frame No: 2300
Frame No: 2400
Frame No: 2500
Frame No: 2600
Frame No: 2700
Frame No: 2800
Frame No: 2900
Frame No: 3000
Frame No: 3100
Frame No: 3200
Frame No: 3300
Frame No: 3400
Frame No: 3500
Frame No: 3600
Frame No: 3700
Frame No: 3800
Frame No: 3900
Frame No: 4000
Frame No: 4100
Frame No: 4200
Frame No: 4300
Frame No: 4400
Frame No: 4500
Frame No: 4600
Frame No: 4700
Frame No: 4800
Frame No: 4900
Frame No: 5000
Frame No: 5100
Frame No: 5200
Frame No: 5300
Frame No: 5400
Frame No: 5500
Frame No: 5600
Frame No: 5700
Frame No: 5800
Frame No: 5900
Frame No: 6000
Frame No: 6100
Frame No: 6200
Frame No: 6300
Frame No: 6400
Frame No: 6500
Frame No: 6600
Frame No: 6700
Fram